# Lab: LangGraph with Tools, Conditional Logic, and Memory

This lab builds on our first LangGraph application by introducing three powerful features that are essential for creating sophisticated agents:

1.  **Tools**: We will create tools from Python functions and give our agent the ability to use them (e.g., to perform a web search or send a notification).
2.  **Conditional Edges**: We will build a more complex graph that can dynamically route its logic. The agent will decide for itself whether to respond directly to the user or to first use a tool.
3.  **Persistent Memory (Checkpoints)**: We will add a checkpointer to our graph, allowing it to save its state after each step. This gives our agent long-term memory, enabling it to recall previous turns in a conversation.

### Setup: External Tool APIs

This lab uses two external services for its tools. You will need to get API keys for them and add them to your `.env` file.

1.  **Serper for Google Search**: Go to [Serper.dev](https://serper.dev/) and sign up for a free account to get an API key.
    ```
    SERPER_API_KEY="YOUR_KEY_HERE"
    ```
2.  **Pushover for Notifications (Optional)**: Go to [Pushover.net](https://pushover.net/) to get a User Key and create an Application to get an API Token.
    ```
    PUSHOVER_USER="YOUR_USER_KEY_HERE"
    PUSHOVER_TOKEN="YOUR_API_TOKEN_HERE"
    ```

In [ ]:
# === Imports ===
import os
import requests
import sqlite3
from typing import Annotated, TypedDict
from dotenv import load_dotenv

# LangChain components for creating tools
from langchain.agents import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_openai import ChatOpenAI

# Core LangGraph components
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# LangGraph components for adding persistent memory
from langgraph.checkpoint.sqlite import SqliteSaver

# For the UI
from IPython.display import Image, display
import gradio as gr

In [ ]:
load_dotenv(override=True)

### Part 1: Creating Tools for Our Agent

A "Tool" is a function that an agent can decide to call. LangChain provides convenient wrappers to easily convert any Python function into a tool that an LLM can understand and use.

In [ ]:
# === Tool 1: Web Search ===
# LangChain Community provides a simple wrapper for the Serper Google Search API.
search_wrapper = GoogleSerperAPIWrapper()

# We wrap the search function in a LangChain `Tool` object.
# This adds a name and description, which the LLM uses to decide when to use the tool.
search_tool = Tool(
    name="search",
    func=search_wrapper.run,
    description="Useful for when you need to answer questions about current events or look up information on the web."
)

In [ ]:
# === Tool 2: Push Notification (Optional) ===
# We can also create a tool from our own custom function.

def send_push_notification(text: str):
    """Send a push notification to the user's device via Pushover."""
    pushover_token = os.getenv("PUSHOVER_TOKEN")
    pushover_user = os.getenv("PUSHOVER_USER")
    if pushover_token and pushover_user:
        requests.post(
            "https://api.pushover.net/1/messages.json", 
            data={"token": pushover_token, "user": pushover_user, "message": text}
        )
        return "Notification sent successfully."
    else:
        return "Pushover credentials not set. Could not send notification."

push_tool = Tool(
    name="send_push_notification",
    func=send_push_notification,
    description="Use this tool to send a push notification to the user."
)

tools = [search_tool, push_tool]

### Part 2: Building a Conditional Graph

Now, we'll build a more advanced graph. Instead of a simple `START -> node -> END` flow, this graph will have a conditional edge. After the chatbot node runs, the graph will check if the LLM decided to call a tool. 
- If YES, it will route to a `ToolNode` to execute the tool.
- If NO, it will route directly to the `END`.

In [ ]:
# Step 1: Define the State
# We'll use a TypedDict for the state this time, which is another common pattern.
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Create the Graph Builder
graph_builder = StateGraph(State)

In [ ]:
# Step 3: Define the Nodes

# First, we bind our tools to the LLM. This tells the LLM that these tools are available for it to call.
llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

# The chatbot node remains simple: it just calls the LLM.
def chatbot_node(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# We also add a `ToolNode`. This is a pre-built node from LangGraph that knows how to execute tools.
tool_node = ToolNode(tools=tools)

graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_node("tools", tool_node)

In [ ]:
# Step 4: Add Edges

# The entry point is the chatbot.
graph_builder.add_edge(START, "chatbot")

# This is the conditional edge. After the 'chatbot' node, it calls `tools_condition`.
# `tools_condition` is a built-in function that checks if the last message contains a tool call.
# If it does, the graph transitions to the 'tools' node. Otherwise, it transitions to END.
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
    # The third argument is a dictionary mapping the condition's outcomes to the next node.
    # In this case, if `tools_condition` returns "tools", we go to the "tools" node.
    # If it returns anything else (like END), we end the graph.
    {
        "tools": "tools",
        "__end__": "__end__"
    }
)

# After the tools are executed, we always loop back to the chatbot node to let it process the tool's output.
graph_builder.add_edge("tools", "chatbot")

### Part 3: Adding Persistent Memory with Checkpoints

By default, a LangGraph's state is ephemeral and resets after each `.invoke()` call. To give our chatbot memory, we need to add a **checkpointer**. The checkpointer saves the state of the graph at every step, allowing us to resume conversations.

We'll use `SqliteSaver` to store the conversation history in a local SQLite database file.

In [ ]:
# The checkpointer connects to a SQLite database file.
# `check_same_thread=False` is required for SQLite in this context.
memory = SqliteSaver.from_conn_string("memory.db")

In [ ]:
# Step 5: Compile the Graph with the Checkpointer
# We pass the memory object to the `.compile()` method.
graph = graph_builder.compile(checkpointer=memory)

display(Image(graph.get_graph().draw_mermaid_png()))

### Showtime! Running the Stateful Chatbot

To use the memory, we need to pass a `config` object to the `.invoke()` call. The `thread_id` in the config tells LangGraph which conversation history to load and save. Each unique `thread_id` will have its own separate memory.

In [ ]:
# We'll use a unique thread_id for this conversation.
# You can change this ID to start a new, separate conversation.
config = {"configurable": {"thread_id": "my-first-thread"}}

def chat_interface_function(user_input: str, history: list):
    # We invoke the graph with the user's message and the config object.
    result = graph.invoke(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config
    )
    # The graph's state is automatically saved and loaded by the checkpointer.
    return result["messages"][-1].content

gr.ChatInterface(
    chat_interface_function, 
    title="LangGraph Agent with Tools and Memory",
    description="Ask me a question that requires a web search, or ask me to send a notification!",
    examples=["What is the latest news on AI?", "Send me a notification that says 'Hello from my agent!'"]
).launch()

In [ ]:
# You can inspect the state history of any conversation thread at any time.
print("--- Conversation History for thread 'my-first-thread' ---")
for state in graph.get_state_history(config):
    print(state.values['messages'][-1])
    print("---")